# Practical Exam: Supermarket Loyalty

International Essentials is an international supermarket chain.

Shoppers at their supermarkets can sign up for a loyalty program that provides rewards each year to customers based on their spending. The more you spend the bigger the rewards. 

The supermarket would like to be able to predict the likely amount customers in the program will spend, so they can estimate the cost of the rewards. 

This will help them to predict the likely profit at the end of the year.

## Data

The dataset contains records of customers for their last full year of the loyalty program.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|


# Task 1

Before you fit any models, you will need to make sure the data is clean. 

The table below shows what the data should look like. 

Create a cleaned version of the dataframe. 

 - You should start with the data in the file "loyalty.csv". 

 - Your output should be a dataframe named `clean_data`. 

 - All column names and values should match the table below.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|customer_id | Unique identifier for the customer. </br>Missing values are not possible due to the database structure. |
|spend | Continuous. </br>The total spend of the customer in their last full year. This can be any positive value to two decimal places. </br>Missing values should be replaced with 0. |
|first_month | Continuous. </br>The amount spent by the customer in their first month of the year. This can be any positive value, rounded to two decimal places. </br>Missing values should be replaced with 0. |
| items_in_first_month | Discrete. </br>The number of items purchased in the first month. Any integer value greater than or equal to zero. </br>Missing values should be replaced by 0. |  
| region | Nominal. </br>The geographic region that the customer is based in. One of four values Americas, Asia/Pacific, Europe, Middle East/Africa. </br>Missing values should be replaced with "Unknown". |
| loyalty_years | Oridinal. </br>The number of years the customer has been a part of the loyalty program. One of five ordered categories, '0-1', '1-3', '3-5', '5-10', '10+'. </br>Missing values should be replaced with '0-1'.|
| joining_month | Nominal. </br>The month the customer joined the loyalty program. One of 12 values "Jan", "Feb", "Mar", "Apr", etc. </br>Missing values should be replaced with "Unknown".|
| promotion | Nominal. </br>Did the customer join the loyalty program as part of a promotion? Either 'Yes' or 'No'. </br>Missing values should be replaced with 'No'.|

In [23]:
# Use this cell to write your code for Task 1

#Task1

# Write your answer to Task 1 here
import numpy as np
import pandas as pd
from datetime import datetime
import re
import os
from IPython.display import FileLink

#reading data
df1=pd.read_csv("loyalty.csv")
column_list = list(df1.columns)
df1.dtypes

customer_id               int64
spend                   float64
first_month              object
items_in_first_month      int64
region                   object
loyalty_years            object
joining_month            object
promotion                object
dtype: object

In [24]:
# Droping duplicates and empty rows in DataFrame

def quick_clean(df):                 #defining function
    return df.dropna().drop_duplicates().reset_index(drop=True)

df1=quick_clean(df1)
df1.shape

(1121, 8)

In [25]:
#defining function to check column null or empty and display summary
column_list = list(df1.columns)
def check_all_columns_empty(df,column_list=None,show_details=True):
     # If no column list provided, use all columns
    if column_list is None:
        column_list = df.columns.tolist()
    results = {}
    for col in column_list:
        if col in df.columns:
            results[col] = {
                'null': df[col].isnull().sum(),
                'empty': (df[col].astype(str).str.strip() == '').sum(),
                'nan': df[col].isna().sum(),
                'special_chars': df[col].astype(str).str.contains(r'[^a-zA-Z0-9\s]', na=False).sum()
            }
        else:
            print(f"Warning: Column '{col}' not found in DataFrame")
    if show_details:
        for col, issues in results.items():
            print(f"{col}: {issues}")
    return results

# Usage
Null_summary= check_all_columns_empty(df1)


customer_id: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 0}
spend: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 1121}
first_month: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 1112}
items_in_first_month: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 0}
region: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 558}
loyalty_years: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 1121}
joining_month: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 0}
promotion: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 0}


In [26]:
#customer_id
# Remove any duplicate customer_id if they exist (keep first occurrence)
df1 = df1.drop_duplicates(subset=['customer_id'], keep='first')
df1 = df1.dropna(subset=['customer_id'])
df1.shape

(1121, 8)

In [27]:
#Spend
#Check if it has negative value
print("Spend has negative values:", (df1['spend'] < 0).any())
#print(df1['spend'].unique())

Spend has negative values: False


In [28]:
#Define function to check and convert negativw values to positive
def convert_to_positive(df, column):
    """Simple function to check if column is numeric and convert negatives to positive"""
    
    # Check if column exists
    if column not in df.columns:
        print(f"Column '{column}' not found")
        return df
    
    # Check if column is numeric (int or float)
    if df[column].dtype in ['int64', 'float64', 'int32', 'float32']:
        negative_count = (df[column] < 0).sum()
        print(f"Column '{column} dtype: {df[column].dtype}")
        if negative_count > 0:
            print(f"Converting {negative_count} negative values to positive in '{column}'")
            df[column] = df[column].abs()
        else:
            print(f"No negative values found in '{column}'")
    else:
        print(f"Column '{column}' is not numeric (int/float)")
    
    return df


In [29]:
#Usage
#Spend
df1 = convert_to_positive(df1, 'spend')
df1['spend']=df1['spend'].round(2)
#df1['spend'].unique()

Column 'spend dtype: float64
No negative values found in 'spend'


In [30]:
#first_month
#Check null,empty nan and convert to numeric and roundoff
df1['first_month']=pd.to_numeric(df1['first_month'], errors='coerce')
#df1['first_month'].dtype
df1=convert_to_positive(df1,'first_month')
df1['first_month'].unique()

#check null
Null_summary= check_all_columns_empty(df1)

Column 'first_month dtype: float64
No negative values found in 'first_month'
customer_id: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 0}
spend: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 1121}
first_month: {'null': 113, 'empty': 0, 'nan': 113, 'special_chars': 1008}
items_in_first_month: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 0}
region: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 558}
loyalty_years: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 1121}
joining_month: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 0}
promotion: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 0}


In [31]:
#first_month
df1['first_month']=df1['first_month'].replace('nan',0).fillna(0).round(2)
#check null
Null_summary= check_all_columns_empty(df1)
#df1['first_month'].unique()

customer_id: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 0}
spend: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 1121}
first_month: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 1121}
items_in_first_month: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 0}
region: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 558}
loyalty_years: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 1121}
joining_month: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 0}
promotion: {'null': 0, 'empty': 0, 'nan': 0, 'special_chars': 0}


In [32]:
#items_in_first_month

df1 = convert_to_positive(df1, 'items_in_first_month')
df1['items_in_first_month']=df1['items_in_first_month']
df1['items_in_first_month']=df1['items_in_first_month'].replace('nan',0).fillna(0)
df1['items_in_first_month'].unique()

Column 'items_in_first_month dtype: int64
No negative values found in 'items_in_first_month'


array([ 5, 14,  7,  8, 13,  9,  6, 12, 11, 10, 15])

In [33]:
#region
df1['region']=df1['region'].replace('nan','Unknown').fillna('Unknown').astype('category')
df1['region'].dtype

CategoricalDtype(categories=['Americas', 'Asia/Pacific', 'Europe', 'Middle East/Africa'], ordered=False)

In [34]:
#loyalty_years
df1['loyalty_years']=df1['loyalty_years'].replace('nan','0-1').fillna('0-1').astype('category')
df1['loyalty_years'].dtype
df1['loyalty_years'].unique()

['5-10', '0-1', '10+', '3-5', '1-3']
Categories (5, object): ['0-1', '1-3', '10+', '3-5', '5-10']

In [35]:
#joining_month
df1['joining_month']=df1['joining_month'].replace('nan','Unknown').fillna('Unknown').str.title().astype('category')
df1['joining_month'].dtype
df1['joining_month'].unique()

['Nov', 'Feb', 'Dec', 'Apr', 'May', ..., 'Jan', 'Sep', 'Mar', 'Jun', 'Aug']
Length: 12
Categories (12, object): ['Apr', 'Aug', 'Dec', 'Feb', ..., 'May', 'Nov', 'Oct', 'Sep']

In [36]:
#promotion
#df1['promotion'].unique()
df1['promotion']=df1['promotion'].replace('nan','No').fillna('No').str.title().astype('category')
df1['promotion'].dtype
df1['promotion'].unique()

['No', 'Yes']
Categories (2, object): ['No', 'Yes']

In [37]:
#check
df1.dtypes

clean_data=df1

# Task 2 

The team at International Essentials have told you that they have always believed that the number of years in the loyalty scheme is the biggest driver of spend. 

Producing a table showing the difference in the average spend by number of years in the loyalty programme along with the variance to investigate this question for the team.

 - You should start with the data in the file 'loyalty.csv'.

 - Your output should be a data frame named `spend_by_years`. 

 - It should include the three columns `loyalty_years`, `avg_spend`, `var_spend`. 

 - Your answers should be rounded to 2 decimal places.   

In [38]:
# Use this cell to write your code for Task 2
# Write your answer to Task 1 here
import numpy as np
import pandas as pd
from datetime import datetime
import re
import os
from IPython.display import FileLink

#reading data
df1=pd.read_csv("loyalty.csv")
column_list = list(df1.columns)
df1.dtypes

customer_id               int64
spend                   float64
first_month              object
items_in_first_month      int64
region                   object
loyalty_years            object
joining_month            object
promotion                object
dtype: object

In [39]:
#groupby
spend_by_years=df1.groupby('loyalty_years').agg(
    avg_spend = ('spend', 'mean'),
    var_spend = ('spend', 'var')
).round(2).reset_index()
print(spend_by_years)

  loyalty_years  avg_spend  var_spend
0           0-1     110.56       9.30
1           1-3     129.31       9.65
2           10+     117.41      16.72
3           3-5     124.55      11.09
4          5-10     135.15      14.10


# Task 3

Fit a baseline model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `base_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [47]:
# Use this cell to write your code for Task 3
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
import warnings
warnings.filterwarnings('ignore')

# Load the data
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

print("Train data shape:", train_df.shape)
print("Test data shape:", test_df.shape)
print("\nTrain columns:", train_df.columns.tolist())
print("Test columns:", test_df.columns.tolist())

# Check for missing values
print("\nMissing values in train:", train_df.isnull().sum())
print("Missing values in test:", test_df.isnull().sum())

# Basic statistics
print("\nTrain data info:")
print(train_df.describe())

Train data shape: (996, 8)
Test data shape: (250, 7)

Train columns: ['customer_id', 'spend', 'first_month', 'items_in_first_month', 'region', 'loyalty_years', 'joining_month', 'promotion']
Test columns: ['customer_id', 'first_month', 'items_in_first_month', 'region', 'loyalty_years', 'joining_month', 'promotion']

Missing values in train: customer_id             0
spend                   0
first_month             0
items_in_first_month    0
region                  0
loyalty_years           0
joining_month           0
promotion               0
dtype: int64
Missing values in test: customer_id             0
first_month             0
items_in_first_month    0
region                  0
loyalty_years           0
joining_month           0
promotion               0
dtype: int64

Train data info:
       customer_id       spend  first_month  items_in_first_month
count   996.000000  996.000000   996.000000            996.000000
mean    624.411647  129.286948    18.722018              9.139558
st

# Task 4

Fit a comparison model to predict the spend over the year for each customer.

 1. Fit your model using the data contained in “train.csv” </br></br>

 2. Use “test.csv” to predict new values based on your model. You must return a dataframe named `compare_result`, that includes `customer_id` and `spend`. The `spend` column must be your predicted values.

In [41]:
# Use this cell to write your code for Task 4